In [2]:
pip install fastapi


  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
Using cached fastapi-0.115.12-py3-none-any.whl (95 kB)
Using cached starlette-0.46.2-py3-none-any.whl (72 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

# Load models
logreg_model = joblib.load("logistic_regression_model.pkl")
scaler = joblib.load("scaler.pkl")

# Initialize app
app = FastAPI(title="Diabetes Prediction API")

# Define input data model
class PatientData(BaseModel):
    pregnancies: float
    glucose: float
    blood_pressure: float
    skin_thickness: float
    insulin: float
    bmi: float
    diabetes_pedigree_function: float
    age: float

# Health check
@app.get("/")
def read_root():
    return {"message": "Diabetes Prediction API is live!"}

# Predict endpoint
@app.post("/predict")
def predict(data: PatientData):
    features = np.array([
        data.pregnancies,
        data.glucose,
        data.blood_pressure,
        data.skin_thickness,
        data.insulin,
        data.bmi,
        data.diabetes_pedigree_function,
        data.age
    ]).reshape(1, -1)

    features_scaled = scaler.transform(features)
    prediction = logreg_model.predict(features_scaled)[0]
    probability = logreg_model.predict_proba(features_scaled)[0][1]

    return {
        "prediction": int(prediction),
        "probability_diabetic": round(float(probability), 4)
    }

